# Dependencies

## Modules

In [19]:
!pip install --no-cache-dir torch transformers datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
!pip install --no-cache-dir seqeval==1.2.2 --use-pep517

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=04db12ed98ecdd50ad826b646fb7e56a2fec009efb6467dcdf66792100b86c45
  Stored in directory: /tmp/pip-ephem-wheel-cache-usdizwj5/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [26]:
!pip install --no-cache-dir optuna

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Imports

In [28]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
from seqeval.metrics import classification_report
import optuna
from sklearn.metrics import f1_score

# Dataset

In [8]:
# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

In [9]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Tokenization and alignment
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(batch["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [10]:
# DataLoader collate function
def collate_fn(batch):
    input_ids = [torch.tensor(x["input_ids"]) for x in batch]
    attention_mask = [torch.tensor(x["attention_mask"]) for x in batch]
    labels = [torch.tensor(x["labels"]) for x in batch]

    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask_padded,
        "labels": labels_padded,
    }

train_loader = DataLoader(tokenized_datasets["train"], batch_size=16, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_datasets["validation"], batch_size=16, collate_fn=collate_fn)

# Model

In [14]:
class BertForNERWithLayerAttention(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(BertForNERWithLayerAttention, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name, output_hidden_states=True)
        self.num_hidden_layers = self.bert.config.num_hidden_layers + 1 
        self.layer_weights = nn.Parameter(torch.ones(self.num_hidden_layers))
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = torch.stack(outputs.hidden_states, dim=0)  # (num_layers, batch_size, seq_len, hidden_size)
        weighted_hidden_states = torch.sum(self.layer_weights[:, None, None, None] * hidden_states, dim=0)
        sequence_output = self.dropout(weighted_hidden_states)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            active_loss = attention_mask.view(-1) == 1
            active_logits = logits.view(-1, logits.size(-1))[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss = loss_fn(active_logits, active_labels)

        return {"loss": loss, "logits": logits}

In [15]:
# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForNERWithLayerAttention(pretrained_model_name="bert-base-cased", num_labels=num_labels).to(device)
# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

## Training

In [16]:
# Training Loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader)}")

Epoch 1: 100%|██████████| 878/878 [01:52<00:00,  7.81it/s]


Epoch 1 Loss: 0.5430636537086109


Epoch 2: 100%|██████████| 878/878 [01:52<00:00,  7.80it/s]


Epoch 2 Loss: 0.13387319764031036


Epoch 3: 100%|██████████| 878/878 [01:52<00:00,  7.78it/s]

Epoch 3 Loss: 0.06589005025081543


## Evaluation

In [25]:
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        preds = torch.argmax(logits, dim=-1)

        for pred, label in zip(preds.cpu().numpy(), labels.cpu().numpy()):
            predictions.append([label_list[p] for p, l in zip(pred, label) if l != -100])
            true_labels.append([label_list[l] for p, l in zip(pred, label) if l != -100])

print(classification_report(true_labels, predictions))

Evaluating: 100%|██████████| 204/204 [00:08<00:00, 25.25it/s]


              precision    recall  f1-score   support

         LOC       0.94      0.94      0.94      3635
        MISC       0.79      0.85      0.82      1480
         ORG       0.92      0.84      0.88      2702
         PER       0.96      0.95      0.95      3329

   micro avg       0.92      0.90      0.91     11146
   macro avg       0.90      0.89      0.90     11146
weighted avg       0.92      0.90      0.91     11146



# Hyperparameter Tuning

## Optuna Study

In [31]:
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_epochs", 2, 5)

    # Define model with trial's hyperparameters
    class BertForNER(nn.Module):
        def __init__(self, pretrained_model_name, num_labels):
            super(BertForNER, self).__init__()
            self.bert = AutoModel.from_pretrained(pretrained_model_name, output_hidden_states=True)
            self.num_hidden_layers = self.bert.config.num_hidden_layers + 1
            self.layer_weights = nn.Parameter(torch.ones(self.num_hidden_layers))
            self.dropout = nn.Dropout(dropout_rate)  # Use trial's dropout rate
            self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

        def forward(self, input_ids, attention_mask, labels=None):
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            hidden_states = torch.stack(outputs.hidden_states, dim=0)  # (num_layers, batch_size, seq_len, hidden_size)
            weighted_hidden_states = torch.sum(self.layer_weights[:, None, None, None] * hidden_states, dim=0)
            sequence_output = self.dropout(weighted_hidden_states)
            logits = self.classifier(sequence_output)

            loss = None
            if labels is not None:
                loss_fn = nn.CrossEntropyLoss()
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, logits.size(-1))[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fn(active_logits, active_labels)

            return {"loss": loss, "logits": logits}

    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertForNER(pretrained_model_name="bert-base-cased", num_labels=num_labels).to(device)

    # Define optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Prepare data loaders
    train_loader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, collate_fn=collate_fn)
    val_loader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size, collate_fn=collate_fn)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs["loss"]
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs["logits"]
            preds = torch.argmax(logits, dim=-1)

            for pred, label in zip(preds.cpu().numpy(), labels.cpu().numpy()):
                predictions.append([label_list[p] for p, l in zip(pred, label) if l != -100])
                true_labels.append([label_list[l] for p, l in zip(pred, label) if l != -100])

    # Compute F1-score
    f1 = f1_score([label for seq in true_labels for label in seq],
                  [label for seq in predictions for label in seq],
                  average="weighted")
    return f1

In [32]:
# Create Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # Try 20 different combinations

# Print best parameters
print("Best hyperparameters:", study.best_params)

[I 2024-12-09 19:42:18,237] A new study created in memory with name: no-name-58a8c8c1-9d49-41fa-a7d6-3522bb4528bd
/tmp/ipykernel_24666/1082680092.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/tmp/ipykernel_24666/1082680092.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.1, 0.5)
Epoch 2:  80%|███████▉  | 700/878 [01:25<00:21,  8.19it/s]
[W 2024-12-09 19:45:33,126] Trial 0 failed with parameters: {'learning_rate': 1.3187782117023046e-05, 'dropout_rate': 0.3402174697038727, 'batch_size': 16, 'num_epochs': 4} because of the following error: KeyboardInterrupt

KeyboardInterrupt: 